In [ ]:
import React, { useState } from 'react';
import { base44 } from '@/api/base44Client';
import { useQuery, useMutation, useQueryClient } from '@tanstack/react-query';
import { motion } from 'framer-motion';
import { toast } from 'sonner';
import { 
  User, Target, Scale, Ruler, Activity, AlertCircle, 
  Flame, Beef, Wheat, Droplets, ChevronRight, Check, Pencil
} from 'lucide-react';
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Badge } from "@/components/ui/badge";
import {
  Sheet,
  SheetContent,
  SheetHeader,
  SheetTitle,
  SheetFooter,
} from "@/components/ui/sheet";
import {
  Select,
  SelectContent,
  SelectItem,
  SelectTrigger,
  SelectValue,
} from "@/components/ui/select";

const dietaryGoals = {
  weight_loss: { label: 'Lose Weight', emoji: '🔥' },
  muscle_gain: { label: 'Build Muscle', emoji: '💪' },
  maintenance: { label: 'Maintain', emoji: '⚖️' },
  health_improvement: { label: 'Eat Healthier', emoji: '🥗' },
};

const dietTypes = {
  none: { label: 'No Preference', emoji: '🍽️' },
  vegan: { label: 'Vegan', emoji: '🌱' },
  vegetarian: { label: 'Vegetarian', emoji: '🥬' },
  keto: { label: 'Keto', emoji: '🥑' },
  paleo: { label: 'Paleo', emoji: '🥩' },
  mediterranean: { label: 'Mediterranean', emoji: '🫒' },
  low_carb: { label: 'Low Carb', emoji: '🥚' },
  diabetic_friendly: { label: 'Diabetic Friendly', emoji: '💚' },
};

const activityLevels = {
  sedentary: { label: 'Sedentary', desc: 'Little to no exercise' },
  light: { label: 'Light', desc: '1-3 days/week' },
  moderate: { label: 'Moderate', desc: '3-5 days/week' },
  active: { label: 'Active', desc: '6-7 days/week' },
  very_active: { label: 'Very Active', desc: 'Intense daily exercise' },
};

const commonAllergens = ['Peanuts', 'Tree Nuts', 'Milk', 'Eggs', 'Wheat', 'Soy', 'Fish', 'Shellfish', 'Sesame'];

export default function ProfilePage() {
  const [editSheet, setEditSheet] = useState(null);
  const [editData, setEditData] = useState({});
  
  const queryClient = useQueryClient();

  const { data: userProfiles = [], isLoading } = useQuery({
    queryKey: ['userProfile'],
    queryFn: () => base44.entities.UserProfile.list(),
  });

  const userProfile = userProfiles[0];

  const updateProfileMutation = useMutation({
    mutationFn: (data) => base44.entities.UserProfile.update(userProfile.id, data),
    onSuccess: () => {
      queryClient.invalidateQueries({ queryKey: ['userProfile'] });
      toast.success('Profile updated!');
      setEditSheet(null);
    },
  });

  const openEdit = (section, data) => {
    setEditData(data);
    setEditSheet(section);
  };

  const handleSave = () => {
    updateProfileMutation.mutate(editData);
  };

  const toggleAllergen = (allergen) => {
    const current = editData.allergies || [];
    setEditData(prev => ({
      ...prev,
      allergies: current.includes(allergen)
        ? current.filter(a => a !== allergen)
        : [...current, allergen]
    }));
  };

  if (isLoading) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center">
        <div className="w-12 h-12 border-4 border-emerald-500 border-t-transparent rounded-full animate-spin" />
      </div>
    );
  }

  if (!userProfile) {
    return (
      <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex items-center justify-center p-4">
        <div className="text-center">
          <User className="w-16 h-16 text-slate-300 mx-auto mb-4" />
          <h2 className="text-xl font-semibold text-slate-800 mb-2">No Profile Yet</h2>
          <p className="text-slate-500">Start by scanning your first meal!</p>
        </div>
      </div>
    );
  }

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 pb-24">
      <div className="max-w-lg mx-auto px-4 py-6">
        {/* Header */}
        <motion.div 
          initial={{ opacity: 0, y: -20 }}
          animate={{ opacity: 1, y: 0 }}
          className="mb-8"
        >
          <h1 className="text-2xl font-bold text-slate-800">Profile</h1>
          <p className="text-slate-500 text-sm mt-1">Manage your nutrition settings</p>
        </motion.div>

        {/* Goals Section */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.1 }}
          className="mb-6"
        >
          <div 
            onClick={() => openEdit('goals', { 
              dietary_goal: userProfile.dietary_goal,
              diet_type: userProfile.diet_type 
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="flex items-center justify-between">
              <div className="flex items-center gap-4">
                <div className="w-12 h-12 rounded-2xl bg-gradient-to-br from-emerald-500 to-teal-600 flex items-center justify-center">
                  <Target className="w-6 h-6 text-white" />
                </div>
                <div>
                  <p className="font-semibold text-slate-800">
                    {dietaryGoals[userProfile.dietary_goal]?.emoji} {dietaryGoals[userProfile.dietary_goal]?.label || 'Set Goal'}
                  </p>
                  <p className="text-sm text-slate-500">
                    {dietTypes[userProfile.diet_type]?.emoji} {dietTypes[userProfile.diet_type]?.label || 'No diet preference'}
                  </p>
                </div>
              </div>
              <ChevronRight className="w-5 h-5 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Daily Targets */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.2 }}
          className="mb-6"
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Daily Targets</h3>
          <div 
            onClick={() => openEdit('targets', {
              daily_calorie_target: userProfile.daily_calorie_target,
              daily_protein_target: userProfile.daily_protein_target,
              daily_carbs_target: userProfile.daily_carbs_target,
              daily_fats_target: userProfile.daily_fats_target,
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="grid grid-cols-2 gap-4">
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-orange-50 flex items-center justify-center">
                  <Flame className="w-5 h-5 text-orange-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_calorie_target || 2000}</p>
                  <p className="text-xs text-slate-500">Calories</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-rose-50 flex items-center justify-center">
                  <Beef className="w-5 h-5 text-rose-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_protein_target || 150}g</p>
                  <p className="text-xs text-slate-500">Protein</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-amber-50 flex items-center justify-center">
                  <Wheat className="w-5 h-5 text-amber-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_carbs_target || 250}g</p>
                  <p className="text-xs text-slate-500">Carbs</p>
                </div>
              </div>
              <div className="flex items-center gap-3">
                <div className="w-10 h-10 rounded-xl bg-blue-50 flex items-center justify-center">
                  <Droplets className="w-5 h-5 text-blue-500" />
                </div>
                <div>
                  <p className="text-lg font-bold text-slate-800">{userProfile.daily_fats_target || 65}g</p>
                  <p className="text-xs text-slate-500">Fats</p>
                </div>
              </div>
            </div>
            <div className="flex justify-end mt-3">
              <Pencil className="w-4 h-4 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Body Stats */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.3 }}
          className="mb-6"
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Body Stats</h3>
          <div 
            onClick={() => openEdit('stats', {
              weight: userProfile.weight,
              height: userProfile.height,
              age: userProfile.age,
              activity_level: userProfile.activity_level,
            })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="grid grid-cols-3 gap-4 text-center">
              <div>
                <Scale className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800">{userProfile.weight || '--'} kg</p>
                <p className="text-xs text-slate-500">Weight</p>
              </div>
              <div>
                <Ruler className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800">{userProfile.height || '--'} cm</p>
                <p className="text-xs text-slate-500">Height</p>
              </div>
              <div>
                <Activity className="w-5 h-5 text-slate-400 mx-auto mb-1" />
                <p className="font-semibold text-slate-800 capitalize">{activityLevels[userProfile.activity_level]?.label || '--'}</p>
                <p className="text-xs text-slate-500">Activity</p>
              </div>
            </div>
          </div>
        </motion.div>

        {/* Allergies */}
        <motion.div
          initial={{ opacity: 0, y: 20 }}
          animate={{ opacity: 1, y: 0 }}
          transition={{ delay: 0.4 }}
        >
          <h3 className="text-sm font-medium text-slate-500 mb-3 px-1">Allergies & Alerts</h3>
          <div 
            onClick={() => openEdit('allergies', { allergies: userProfile.allergies || [] })}
            className="bg-white rounded-2xl p-5 border border-slate-100 shadow-sm cursor-pointer hover:border-slate-200 transition-colors"
          >
            <div className="flex items-start gap-3">
              <AlertCircle className="w-5 h-5 text-red-500 mt-0.5" />
              <div className="flex-1">
                {userProfile.allergies?.length > 0 ? (
                  <div className="flex flex-wrap gap-2">
                    {userProfile.allergies.map(allergen => (
                      <Badge key={allergen} variant="secondary" className="bg-red-50 text-red-700 border-red-200">
                        {allergen}
                      </Badge>
                    ))}
                  </div>
                ) : (
                  <p className="text-slate-500">No allergies set</p>
                )}
              </div>
              <ChevronRight className="w-5 h-5 text-slate-400" />
            </div>
          </div>
        </motion.div>

        {/* Edit Sheets */}
        <Sheet open={editSheet === 'goals'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Goals</SheetTitle>
            </SheetHeader>
            <div className="py-6 space-y-4">
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Dietary Goal</label>
                <Select value={editData.dietary_goal} onValueChange={(v) => setEditData(p => ({ ...p, dietary_goal: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select goal" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(dietaryGoals).map(([key, { label, emoji }]) => (
                      <SelectItem key={key} value={key}>{emoji} {label}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Diet Type</label>
                <Select value={editData.diet_type} onValueChange={(v) => setEditData(p => ({ ...p, diet_type: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select diet" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(dietTypes).map(([key, { label, emoji }]) => (
                      <SelectItem key={key} value={key}>{emoji} {label}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'targets'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Daily Targets</SheetTitle>
            </SheetHeader>
            <div className="py-6 grid grid-cols-2 gap-4">
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Calories</label>
                <Input 
                  type="number" 
                  value={editData.daily_calorie_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_calorie_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Protein (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_protein_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_protein_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Carbs (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_carbs_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_carbs_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Fats (g)</label>
                <Input 
                  type="number" 
                  value={editData.daily_fats_target || ''} 
                  onChange={(e) => setEditData(p => ({ ...p, daily_fats_target: parseInt(e.target.value) }))}
                  className="h-12 rounded-xl"
                />
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'stats'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Body Stats</SheetTitle>
            </SheetHeader>
            <div className="py-6 space-y-4">
              <div className="grid grid-cols-3 gap-4">
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Weight (kg)</label>
                  <Input 
                    type="number" 
                    value={editData.weight || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, weight: parseFloat(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Height (cm)</label>
                  <Input 
                    type="number" 
                    value={editData.height || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, height: parseFloat(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
                <div>
                  <label className="text-sm font-medium text-slate-600 mb-2 block">Age</label>
                  <Input 
                    type="number" 
                    value={editData.age || ''} 
                    onChange={(e) => setEditData(p => ({ ...p, age: parseInt(e.target.value) }))}
                    className="h-12 rounded-xl"
                  />
                </div>
              </div>
              <div>
                <label className="text-sm font-medium text-slate-600 mb-2 block">Activity Level</label>
                <Select value={editData.activity_level} onValueChange={(v) => setEditData(p => ({ ...p, activity_level: v }))}>
                  <SelectTrigger className="h-12 rounded-xl">
                    <SelectValue placeholder="Select activity" />
                  </SelectTrigger>
                  <SelectContent>
                    {Object.entries(activityLevels).map(([key, { label, desc }]) => (
                      <SelectItem key={key} value={key}>{label} - {desc}</SelectItem>
                    ))}
                  </SelectContent>
                </Select>
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>

        <Sheet open={editSheet === 'allergies'} onOpenChange={() => setEditSheet(null)}>
          <SheetContent side="bottom" className="rounded-t-3xl">
            <SheetHeader>
              <SheetTitle>Edit Allergies</SheetTitle>
            </SheetHeader>
            <div className="py-6">
              <p className="text-sm text-slate-500 mb-4">Tap to toggle allergens you want to be alerted about</p>
              <div className="flex flex-wrap gap-2">
                {commonAllergens.map(allergen => (
                  <Badge
                    key={allergen}
                    variant={editData.allergies?.includes(allergen) ? 'default' : 'outline'}
                    className={`px-4 py-2 text-sm cursor-pointer transition-all ${
                      editData.allergies?.includes(allergen)
                        ? 'bg-red-500 hover:bg-red-600 border-red-500'
                        : 'hover:border-slate-400'
                    }`}
                    onClick={() => toggleAllergen(allergen)}
                  >
                    {editData.allergies?.includes(allergen) && (
                      <Check className="w-3 h-3 mr-1" />
                    )}
                    {allergen}
                  </Badge>
                ))}
              </div>
            </div>
            <SheetFooter>
              <Button onClick={handleSave} className="w-full h-12 rounded-xl bg-emerald-500 hover:bg-emerald-600">
                Save Changes
              </Button>
            </SheetFooter>
          </SheetContent>
        </Sheet>
      </div>
    </div>
  );
}